In [4]:
import pandas as pd
import scipy.stats as stats

# Load the Excel file
file_path = "Consolidado Final.xlsx"
sheet_name = "Só 6 semanas"

df_raw = pd.read_excel(file_path, sheet_name=sheet_name)

# Extract relevant columns for PLT and SSA
df_cleaned = pd.DataFrame({
    "Grupo": ["PLT"] * len(df_raw) + ["SSA"] * len(df_raw),
    "Peso Médio": list(df_raw["Peso"]) + list(df_raw["Unnamed: 3"]),
    "Conversão alimentar": list(df_raw["CA Ponderado"]) + list(df_raw["Unnamed: 12"]),
    "Lucro": list(df_raw["Lucro (R$/kg)"]) + list(df_raw["Unnamed: 18"])
})

# Remove non-numeric rows (headers, empty values, etc.)
df_cleaned = df_cleaned[pd.to_numeric(df_cleaned["Peso Médio"], errors='coerce').notna()]

# Convert values to numeric types
df_cleaned[["Peso Médio", "Conversão alimentar", "Lucro"]] = df_cleaned[["Peso Médio", "Conversão alimentar", "Lucro"]].apply(pd.to_numeric)

# Split data into PLT and SSA groups
plt_data = df_cleaned[df_cleaned["Grupo"] == "PLT"]
ssa_data = df_cleaned[df_cleaned["Grupo"] == "SSA"]

# Define function for t-test and F-test
def hypothesis_tests(var_name):
    plt_values = plt_data[var_name].dropna()
    ssa_values = ssa_data[var_name].dropna()
    
    # T-test (assuming unequal variance)
    t_stat, t_pvalue = stats.ttest_ind(plt_values, ssa_values, equal_var=False)
    
    # F-test (variance comparison)
    f_stat = plt_values.var() / ssa_values.var()
    df1, df2 = len(plt_values) - 1, len(ssa_values) - 1
    f_pvalue = 1 - stats.f.cdf(f_stat, df1, df2)
    
    return {"t_stat": t_stat, "t_pvalue": t_pvalue, "f_stat": f_stat, "f_pvalue": f_pvalue}

# Run tests on the three variables
results = {}
for var in ["Lucro", "Conversão alimentar", "Peso Médio"]:
    results[var] = hypothesis_tests(var)

# Display results
for var, res in results.items():
    print(f"Results for {var}:")
    print(f"  T-test: t-stat={res['t_stat']:.4f}, p-value={res['t_pvalue']:.4f}")
    print(f"  F-test: F-stat={res['f_stat']:.4f}, p-value={res['f_pvalue']:.4f}\n")


Results for Lucro:
  T-test: t-stat=2.1675, p-value=0.0417
  F-test: F-stat=0.9568, p-value=0.5358

Results for Conversão alimentar:
  T-test: t-stat=-0.9119, p-value=0.3718
  F-test: F-stat=1.2619, p-value=0.3612

Results for Peso Médio:
  T-test: t-stat=0.0673, p-value=0.9470
  F-test: F-stat=2.3259, p-value=0.0953



Avaliação dos resultados dos testes.

### **1. Teste T (Comparação das Médias)**  
O teste t verifica se as médias dos grupos **PLT** e **SSA** são significativamente diferentes.  

- **Lucro (R$/kg)**:  
  - Estatística t = **2.1675**, p-valor = **0.0417**  
  - Como **p-valor < 0.05**, **rejeitamos a hipótese nula**, indicando **diferença significativa no Lucro entre PLT e SSA**.  
  - Isso significa que **um dos grupos tem um lucro médio por kg significativamente maior**.  

- **Conversão alimentar**:  
  - Estatística t = **-0.9119**, p-valor = **0.3718**  
  - Como p-valor > 0.05, **não há diferença estatisticamente significativa**.  
  - Isso indica que **a eficiência alimentar de PLT e SSA é semelhante**.  

- **Peso Médio**:  
  - Estatística t = **0.0673**, p-valor = **0.9470**  
  - Como p-valor > 0.05, **não há diferença estatisticamente significativa**.  
  - **Os pesos médios dos grupos são praticamente iguais**.  

### **2. Teste F (Comparação das Variâncias)**  
O teste F avalia se as variações dos grupos são significativamente diferentes.  

- **Lucro (R$/kg)**: F = **0.9568**, p-valor = **0.5358**  
  - O alto p-valor indica que **as variâncias são semelhantes** entre os grupos.  

- **Conversão alimentar**: F = **1.2619**, p-valor = **0.3612**  
  - **Sem diferença significativa nas variações de Conversão Alimentar**.  

- **Peso Médio**: F = **2.3259**, p-valor = **0.0953**  
  - **Tendência de diferença na variação do Peso Médio**, mas **não estatisticamente significativa** (p > 0.05).  

### **Conclusão**  
- O **Lucro (R$/kg) é significativamente diferente** entre **PLT e SSA**, indicando que um dos grupos é mais lucrativo.  
- **Conversão alimentar e Peso Médio não apresentam diferenças estatísticas**, sugerindo que ambos os grupos têm desempenhos semelhantes nesses aspectos.  
- As variâncias são semelhantes para **Lucro e Conversão alimentar**, enquanto o **Peso Médio pode ter uma variação um pouco maior**.  

Conclusão pessoal: É necessário mais dados e mais análises para pode afirmar estatisticamente as diferenças entre os dois conjuntos de dados.

In [8]:
from fpdf import FPDF
# Generate PDF report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", style='', size=12)
pdf.cell(200, 10, "Resultados dos Testes de Hipótese", ln=True, align='C')

pdf.ln(10)
pdf.multi_cell(0, 10, "1. Teste T (Comparação das Médias)\nO teste t verifica se as médias dos grupos PLT e SSA são significativamente diferentes.\n\nLucro (R$/kg):\n\nEstatística t = 2.1675, p-valor = 0.0417\nComo p-valor < 0.05, rejeitamos a hipótese nula, indicando diferença significativa no Lucro entre PLT e SSA.\nIsso significa que um dos grupos tem um lucro médio por kg significativamente maior.\n\nConversão alimentar:\n\nEstatística t = -0.9119, p-valor = 0.3718\nComo p-valor > 0.05, não há diferença estatisticamente significativa.\nIsso indica que a eficiência alimentar de PLT e SSA é semelhante.\n\nPeso Médio:\n\nEstatística t = 0.0673, p-valor = 0.9470\nComo p-valor > 0.05, não há diferença estatisticamente significativa.\nOs pesos médios dos grupos são praticamente iguais.\n\n2. Teste F (Comparação das Variâncias)\nO teste F avalia se as variações dos grupos são significativamente diferentes.\n\nLucro (R$/kg): F = 0.9568, p-valor = 0.5358\n\nO alto p-valor indica que as variâncias são semelhantes entre os grupos.\n\nConversão alimentar: F = 1.2619, p-valor = 0.3612\n\nSem diferença significativa nas variações de Conversão Alimentar.\n\nPeso Médio: F = 2.3259, p-valor = 0.0953\n\nTendência de diferença na variação do Peso Médio, mas não estatisticamente significativa (p > 0.05).\n\nConclusão\nO Lucro (R$/kg) é significativamente diferente entre PLT e SSA, indicando que um dos grupos é mais lucrativo.\nConversão alimentar e Peso Médio não apresentam diferenças estatísticas, sugerindo que ambos os grupos têm desempenhos semelhantes nesses aspectos.\nAs variâncias são semelhantes para Lucro e Conversão alimentar, enquanto o Peso Médio pode ter uma variação um pouco maior.")

pdf.output("Resultados_Hipotese.pdf")



''